# This section is a Work in Progress

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import urllib
import os

df = pd.read_csv('grammy_winners.csv')

In [6]:
r = requests.get('https://timelines.latimes.com/grammy-awards/')
years = []
songs = []
records = []
for year in range(2018, 1959, -1):
    years.append(year)
soupy = BeautifulSoup(r.text).find_all(attrs={'class': 'tlog-item-description'})
for num in range(0, len(soupy)):
    top_song = str(soupy[num]).split('</b>')[3].split('<b>')[0].replace('<span class="amp">&amp;</span> ', 'and ')[2:-5]
    top_record = str(soupy[num]).split('</b>')[1].split('<b>')[0].replace('<span class="amp">&amp;</span> ', 'and ')[2:-5]
    songs.append(top_song)
    records.append(top_record)
df = pd.DataFrame([years, songs, records]).T
df.rename(columns={0: 'year', 1:'top_song', 2:'top_record'}, inplace=True)
df

In [7]:
df.to_csv(r'grammy_winners.csv')

In [8]:
soup = BeautifulSoup(requests.get('https://www.grammy.com/grammys/awards/61st-annual-grammy-awards-2018').text)
table = soup.find(attrs={'class': 'view-grouping-content'})
songs = [song.text for song in table.find_all(attrs={'class': 'views-field-title'})]
names = [name.text.replace('\n', '') for name in table.find_all(attrs={'class': 'views-field-field-description'})]

entry = str(BeautifulSoup(r.text).find_all(attrs={'class': 'tlog-item-description'})[1].find_all('p')[1]).split('<br/>')[2].split('>')[2][2:].split(',')
title = entry[0]
artist = entry[1][2:]
writer = entry[2].split('(')[1][0:-1]
print('title:', title, '\nartist:', artist, '\nwriter:', writer)

entry = BeautifulSoup(r.text).find_all(attrs={'class': 'tlog-item-description'})[0]
# .find_all('p')[1]).split('<br/>')[2].split('>')[2][2:].split(',')
title = entry[0]
artist = entry[1][2:]
writer = entry[2].split('(')[1][0:-1]
print('title:', title, '\nartist:', artist, '\nwriter:', writer)

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.environ["SPOTIPY_CLIENT_ID"], client_secret=os.environ["SPOTIPY_CLIENT_SECRET"]))

results = sp.search(q='Hello,” Adele', limit=2)
# for track in results:
#     print(track)

In [22]:
results['tracks']['items'][0]['id']

'62PaSfnXSMyLshYJrlTuL3'

In [28]:
df['top_song'][0].split(',')[0]

'Thinking Out Loud'

In [29]:
urllib.parse.urlencode({'query': df['top_song'][0].split(',')[0], 'offset': 0, 'limit': 1, 'type': 'track'})

'query=Thinking+Out+Loud&offset=0&limit=1&type=track'

In [ ]:
get_genius_api_results = False # MARK AS FALSE UNLESS YOU WANT TO MAKE 10k API CALLS

if get_genius_api_results:
    
    print('GETTING API RESULTS - INTERRUPT KERNEL IF THIS IS NOT INTENDED')
    
    # Create a dataframe in which we will pass API results
    df_genius_api = pd.DataFrame(columns=['search_term', 'annotation_count', 'full_title', 'pyongs_count', 'stats_unreviewed_annotations', 'stats_hot', 'song_url', 'primary_artist'])
    
    # Create a function to get data from the Genius API, such as user interaction like views, annotations
    for search_term in df['title_artist_search_term'][0:10000]:
        url = f"https://api.genius.com/search?q={search_term}"
        payload={}
        headers = {
          'Authorization': f'Bearer {os.environ.get("SPOTIFY_API_TOKEN_V2")}',
          'Cookie': os.environ.get("SPOTIFY_API_COOKIE")
        }
        response = requests.request("GET", url, headers=headers, data=payload)
        result_big = json.loads(response.text)['response']['hits']
        if len(result_big) > 0:
            result = result_big[0]['result']
            arr = {'search_term': search_term, 'annotation_count': result['annotation_count'], 'full_title': result['full_title'], 'pyongs_count': result['pyongs_count'], 'stats_unreviewed_annotations': result['stats']['unreviewed_annotations'], 'stats_hot': result['stats']['hot'], 'song_url': result['url'], 'primary_artist': result['primary_artist']['name']}
            df_genius_api = df_genius_api.append(arr, ignore_index=True)
        else:
            arr = {'search_term': search_term, 'annotation_count': 'n/a', 'full_title': 'n/a', 'pyongs_count': 'n/a', 'stats_unreviewed_annotations': 'n/a', 'stats_hot': 'n/a', 'song_url': 'n/a', 'primary_artist': 'n/a'}
            df_genius_api = df_genius_api.append(arr, ignore_index=True)
        time.sleep(.5)
        
    df_genius_api.to_csv(r'genius_api_results.csv')
else:
    print('Reading in df from csv')
    df_genius_api = pd.read_csv('genius_api_results.csv')
print('done')

In [46]:
df['top_record'][0].replace('/"', '-')

'Hello,” Adele'

In [31]:
import requests
url = "https://api.spotify.com/v1/search?query=Thinking+Out+Loud&offset=0&limit=1&type=track"
payload={}
headers = {

}
response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)

{
  "tracks" : {
    "href" : "https://api.spotify.com/v1/search?query=Thinking+Out+Loud&type=track&offset=0&limit=1",
    "items" : [ {
      "album" : {
        "album_type" : "album",
        "artists" : [ {
          "external_urls" : {
            "spotify" : "https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V"
          },
          "href" : "https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V",
          "id" : "6eUKZXaKkcviH0Ku9w2n3V",
          "name" : "Ed Sheeran",
          "type" : "artist",
          "uri" : "spotify:artist:6eUKZXaKkcviH0Ku9w2n3V"
        } ],
        "available_markets" : [ "AE", "AL", "AR", "AT", "AU", "BA", "BE", "BG", "BH", "BO", "BR", "BY", "CA", "CH", "CL", "CO", "CR", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GR", "GT", "HK", "HN", "HR", "HU", "ID", "IE", "IL", "IN", "IT", "JO", "JP", "KR", "KW", "KZ", "LB", "LT", "LU", "LV", "MA", "MD", "ME", "MK", "MT", "MX", "MY", "NI", "NL", "NO", "NZ", "OM", "PA",